In [ ]:
import requests
import json

url = "https://api.sportradar.com/tennis/trial/v3/en/double_competitors_rankings.json?api_key=7TOzdGr7SikuNauga42FVUh84HA0ZS1mvb3AaF3j"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

data=json.loads(response.text)

doublecomrank = data['rankings']

competitor_ranking_list = []
for dubcomp in doublecomrank:
    for rank in dubcomp.get("competitor_rankings", []):
        comp = rank.get("competitor", [])
        competitor_ranking_list.append({
            "rank" : rank["rank"],
            "movement" : rank["movement"],
            "points" : rank["points"],
            "competitions_played" : rank["competitions_played"],
            "id" : comp["id"] 
        })


competitors_list = []
for dubcomp in doublecomrank:
    for rank in dubcomp.get("competitor_rankings", []):
        comp = rank.get("competitor", [])
        competitors_list.append({
            "id" : comp["id"],
            "name" : comp["name"],
            "country" : comp.get("country"),
            "country_code" : comp.get("country_code"),
            "abbreviation" : comp.get("abbreviation"),
            "week" : dubcomp["week"],
            "Year" : dubcomp["year"]
        })



In [ ]:
import pandas as pd
competitor_ranking_df = pd.DataFrame(competitor_ranking_list)
competitor_ranking_df

In [ ]:
competitors_df = pd.DataFrame(competitors_list)
competitors_df

In [ ]:
competitor_ranking_d= competitor_ranking_df.drop_duplicates()
print(competitor_ranking_d)

In [ ]:
competitor_ranking_d.to_csv("competitor_ranking.csv")

In [ ]:
competitors_d= competitors_df.drop_duplicates()
print(competitors_d)

In [ ]:
competitors_d.to_csv("competitors.csv")

In [ ]:
import mysql.connector

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="tennis_data"   
)

mycursor = mydb.cursor(buffered=True) 

In [ ]:
mycursor.execute("""
CREATE TABLE Competitors (
    competitor_id VARCHAR(50) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    country VARCHAR(100) NOT NULL,
    country_code CHAR(3) NOT NULL,
    abbreviation VARCHAR(10) NOT NULL
)
""")

In [ ]:
mycursor.execute("""
CREATE TABLE Competitor_rankings (
    rank_id INT PRIMARY KEY AUTO_INCREMENT,
    rank INT NOT NULL,
    movement INT NOT NULL,
    points INT NOT NULL,
    competitions_played INT NOT NULL,
    competitor_id VARCHAR(50), FOREIGN KEY(competitor_id)
    REFERENCES Competitors(competitor_id)
)
""")

In [ ]:
from tabulate import tabulate

In [ ]:
Competitors_query = ("INSERT INTO Competitors (competitor_id,name,country,country_code,abbreviation,week,year) VALUES(%s, %s,%s, %s, %s, %s, %s)")

Competitors_data = competitors_d[['id','name','country','country_code','abbreviation','week','Year']].values.tolist()

mycursor.executemany(Competitors_query,Competitors_data)
mydb.commit()

In [ ]:
mycursor.execute("SELECT COUNT(*) FROM competitors")
out=mycursor.fetchall()
print(tabulate(out, headers=[i[0] for i in mycursor.description],  tablefmt='psql'))


In [ ]:
mycursor.execute("SELECT * FROM competitors")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
Competitor_rankings_querey = ("INSERT INTO Competitor_rankings (rank,movement,points,competitions_played,competitor_id) VALUES(%s, %s, %s, %s, %s)")

Competitor_rankings_data = competitor_ranking_d[['rank','movement','points','competitions_played','id']].values.tolist()

mycursor.executemany(Competitor_rankings_querey, Competitor_rankings_data)
mydb.commit()

In [ ]:
mycursor.execute("SELECT COUNT(*) FROM Competitor_rankings")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
mycursor.execute("SELECT * FROM Competitor_rankings")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
# Get all competitors with their rank and points

mycursor.execute(""" 
SELECT c.competitor_id,c.name,c.country,r.rank,r.points 
FROM competitors c
JOIN Competitor_rankings r ON r.competitor_id = c.competitor_id
ORDER BY r.points DESC""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))


In [ ]:
# Find competitors ranked in the top 5

mycursor.execute("""
SELECT c.competitor_id,c.name,r.rank
FROM competitors c
JOIN Competitor_rankings r ON r.competitor_id = c.competitor_id
ORDER BY r.rank
LIMIT 5
""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description],tablefmt='psql'))

In [ ]:
# List competitors with no rank movement (stable rank)

mycursor.execute("""
SELECT c.competitor_id,c.name,c.country,r.movement
FROM competitors c
JOIN Competitor_rankings r ON r.competitor_id = c.competitor_id
WHERE r.movement=0
ORDER BY c.country,c.name
""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))


In [ ]:
# Get the total points of competitors from a specific country (e.g., Croatia)

mycursor.execute("""
SELECT c.country,sum(r.points) AS total_points
FROM Competitor_rankings r
JOIN competitors c ON r.competitor_id = c.competitor_id 
WHERE c.country='Italy'
GROUP BY c.country
""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
# Count the number of competitors per country

mycursor.execute("""
SELECT country,COUNT(*) AS count_of_competitors_in_country
FROM competitors
GROUP BY country 
ORDER BY count_of_competitors_in_country DESC
""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))

In [ ]:
# Find competitors with the highest points in the current week
mycursor.execute(""" 
SELECT c.name, c.country, r.points
FROM Competitors c
JOIN Competitor_rankings r ON c.competitor_id = r.competitor_id
WHERE c.week = 16 AND c.Year = 2025
ORDER BY r.points DESC
# LIMIT 1;
""")
out=mycursor.fetchall()
print(tabulate(out,headers=[i[0] for i in mycursor.description], tablefmt='psql'))